In [12]:
import requests
import pandas as pd

def fetch_sg_traffic_flow(api_key):
    # Fetches Singapore traffic flow data using the LTA DataMall API.
    url = 'https://datamall2.mytransport.sg/ltaodataservice/TrafficFlow'

    headers = {
        'AccountKey': api_key,
        'accept': 'application/json'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()

        if 'value' in data and len(data['value']) > 0:
            download_link = data['value'][0].get('Link')
            if download_link:
                download_response = requests.get(download_link)
                download_response.raise_for_status()
                traffic_flow_data = download_response.json()
                df = pd.DataFrame(traffic_flow_data)
                # Flatten the 'Value' column which contains dictionaries
                df_traffic_flow = pd.concat([df.drop(['Value'], axis=1), df['Value'].apply(pd.Series)], axis=1)
                return df_traffic_flow

            else:
                print("No download link found in the response.")
        else:
            print("No data available in the response.")

    except requests.RequestException as e:
        print(f"An error occurred: {e}")

    return None

# Usage
api_key_lta = 'insert_api_key_here'  # Use ltadatamall key
df_traffic_flow = fetch_sg_traffic_flow(api_key_lta)
df_traffic_flow.head(2)
df_traffic_flow.to_csv("sg_traffic_flow_df.csv",index=False)

,LastUpdatedDate,LinkID,Date,HourOfDate,Volume,StartLon,StartLat,EndLon,EndLat,RoadName,RoadCat
0,2024-05-31,103000122,01/02/2024,7,3881,103.694408899999999,1.350438528000000,103.6953675,1.351780186,PAN ISLAND EXPRESSWAY,CAT1
1,2024-05-31,103000122,01/02/2024,8,4644,103.694408899999999,1.350438528000000,103.6953675,1.351780186,PAN ISLAND EXPRESSWAY,CAT1
